In [ ]:
from util.DNN import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import util.Calculations as calc

In [2]:
data = pd.read_csv("../data/eur_usd-hourly.csv", parse_dates=["time"], index_col="time")
data

,price
time,
2015-05-15 13:00:00,1.13546
2015-05-15 14:00:00,1.14378
2015-05-15 15:00:00,1.14282
2015-05-15 16:00:00,1.14320
2015-05-15 17:00:00,1.14368
...,...
2023-08-01 10:00:00,1.09822
2023-08-01 11:00:00,1.09756
2023-08-01 12:00:00,1.09565


In [3]:
df = data.copy()
df["returns"] = calc.returns(df.price)
df["dir"] = calc.dir(df.returns)
df["sma"] = calc.sma_crossover(df.price)
df["mean_reversion"] = calc.mean_reversion(df.price)
df["min"] = calc.min(df.price)
df["max"] = calc.min(df.price)
df["mom"] = calc.momentum(df.returns)
df["vol"] = calc.volume(df.returns)
df["macd"] = calc.macd(df.price)
df["rsi"] = calc.rsi(df.price)
df.dropna(inplace=True)
df

,price,returns,dir,sma,mean_reversion,min,max,mom,vol,macd,rsi
time,,,,,,,,,,,
2015-05-27 17:00:00,1.09007,0.001735,1,-0.020224,-0.153369,-0.006321,-0.006321,0.000661,0.001364,0.000318,52.938222
2015-05-27 18:00:00,1.08896,-0.001019,0,-0.020162,-0.413303,-0.005308,-0.005308,0.000064,0.001365,0.000349,45.388223
2015-05-27 19:00:00,1.09011,0.001055,1,-0.020050,-0.061686,-0.006357,-0.006357,0.000591,0.001373,0.000432,45.331247
2015-05-27 20:00:00,1.09038,0.000248,1,-0.019940,0.061008,-0.006603,-0.006603,0.000095,0.001374,0.000486,46.090535
2015-05-27 21:00:00,1.09006,-0.000294,0,-0.019828,0.012062,-0.006312,-0.006312,0.000337,0.001374,0.000480,45.121951
...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 10:00:00,1.09822,0.000619,1,-0.005686,-1.479394,-0.000783,-0.000783,0.000073,0.000724,-0.000119,37.888199
2023-08-01 11:00:00,1.09756,-0.000601,0,-0.005562,-1.851224,-0.000073,-0.000073,0.000024,0.000707,-0.000117,33.618234
2023-08-01 12:00:00,1.09565,-0.001742,0,-0.005509,-2.662682,0.000000,0.000000,-0.000575,0.000700,-0.000224,24.221453


In [ ]:
lags=10
cols = []
features = [
    "returns",
    "dir",
    "sma",
    "mean_reversion",
    "min",
    "max",
    "mom",
    "vol",
    "macd",
    "rsi",
]

for f in features:
    for lag in range(1, lags + 1):
        col = "{}_lag_{}".format(f, lag)
        df[col] = df[f].shift(lag)
        cols.append(col)
df.dropna(inplace=True)

In [5]:
split = int(len(df) * 0.8)
train = df.iloc[:split].copy()
test = df.iloc[split:].copy()

In [6]:
mean, std = train.mean(), train.std()

In [7]:
train_s = (train - mean) / std
train_s.describe()

,price,returns,dir,sma,mean_reversion,min,max,mom,vol,macd,...,rsi_lag_1,rsi_lag_2,rsi_lag_3,rsi_lag_4,rsi_lag_5,rsi_lag_6,rsi_lag_7,rsi_lag_8,rsi_lag_9,rsi_lag_10
count,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,...,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04,4.126400e+04
mean,1.634124e-15,-1.222580e-17,5.712547e-17,-1.808041e-18,6.198996e-18,-8.050086e-17,-8.050086e-17,9.556786e-18,3.027177e-16,1.954406e-17,...,2.870480e-16,7.748745e-19,4.834356e-16,1.570412e-16,3.538594e-17,1.988845e-16,-1.489481e-17,3.567006e-16,-2.305682e-16,2.667290e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.299029e+00,-2.090006e+01,-9.873211e-01,-4.757089e+00,-4.608068e+00,-9.278851e+00,-9.278851e+00,-1.671091e+01,-1.736061e+00,-1.377121e+01,...,-2.854505e+00,-2.854471e+00,-2.854556e+00,-2.854639e+00,-2.854638e+00,-2.854641e+00,-2.854649e+00,-2.854636e+00,-2.854623e+00,-2.854622e+00
25%,-7.078918e-01,-4.049641e-01,-9.873211e-01,-6.513422e-01,-7.856459e-01,-4.328157e-01,-4.328157e-01,-4.203742e-01,-6.253807e-01,-5.195369e-01,...,-7.398842e-01,-7.399174e-01,-7.399068e-01,-7.398230e-01,-7.398217e-01,-7.398239e-01,-7.398290e-01,-7.398164e-01,-7.398030e-01,-7.398011e-01
50%,-1.816804e-01,-9.098632e-04,-9.873211e-01,-4.575556e-03,-9.917158e-03,2.649554e-01,2.649554e-01,3.624676e-03,-2.034633e-01,2.505346e-02,...,2.843546e-03,2.888077e-03,2.951796e-03,2.967156e-03,2.968565e-03,2.966811e-03,2.962622e-03,2.923610e-03,2.821283e-03,2.823606e-03
75%,8.065289e-01,4.085357e-01,1.012817e+00,6.009090e-01,7.869484e-01,7.297894e-01,7.297894e-01,4.222853e-01,3.216170e-01,5.407646e-01,...,7.363914e-01,7.365251e-01,7.365648e-01,7.365406e-01,7.365421e-01,7.365407e-01,7.365375e-01,7.365500e-01,7.365637e-01,7.365663e-01
max,2.511671e+00,1.615059e+01,1.012817e+00,4.441664e+00,4.517882e+00,1.011971e+00,1.011971e+00,1.462509e+01,8.584707e+00,1.038271e+01,...,2.843816e+00,2.843669e+00,2.843649e+00,2.843660e+00,2.843661e+00,2.843661e+00,2.843661e+00,2.843673e+00,2.843687e+00,2.843691e+00


In [8]:
set_seeds(100)
model = create_model(hl=3, hu=50, dropout=True, input_dim=len(cols))
model.fit(x=train_s[cols], y=train["dir"],epochs=150, verbose=False,validation_split=0.2, shuffle=False,class_weight=cw(train))

In [9]:
model.evaluate(train_s[cols], train["dir"])

1290/1290 [==============================] - 3s 2ms/step - loss: 0.6796 - accuracy: 0.5694


[0.6795790195465088, 0.5694067478179932]

In [10]:
model.save("DNN_model")

INFO:tensorflow:Assets written to: DNN_model/assets


INFO:tensorflow:Assets written to: DNN_model/assets


In [11]:
import pickle
params={"mean": mean, "std": std}
pickle.dump(params,open("params.pkl", "wb"))